## Cut Callback

The cutting plane method has some limitations: even though the first rounds of cuts improve significantly the lower bound, the overall number of iterations needed to obtain the optimal integer solution may be too large. Better results can be obtained with the Branch-&-Cut algorithm, where cut generation is combined with branching. If you have an algorithm like the one included in the previous Section to separate inequalities for your application you can combine it with the complete BC algorithm implemented in the solver engine using callbacks. Cut generation callbacks (CGC) are called at each node of the search tree where a fractional solution is found. Cuts are generated in the callback and returned to the MIP solver engine which adds these cuts to the Cut Pool. These cuts are merged with the cuts generated with the solver builtin cut generators and a subset of these cuts is included to the LP relaxation model. Please note that in the Branch-&-Cut algorithm context cuts are optional components and only those that are classified as good cuts by the solver engine will be accepted, i.e., cuts that are too dense and/or have a small violation could be discarded, since the cost of solving a much larger linear program may not be worth the resulting bound improvement.

When using cut callbacks be sure that cuts are used only to improve the LP relaxation but not to define feasible solutions, which need to be defined by the initial formulation. In other words, the initial model without cuts may be weak but needs to be complete 1. In the case of TSP, we can include the weak sub-tour elimination constraints presented in tsp-label in the initial model and then add the stronger sub-tour elimination constraints presented in the previous section as cuts.

In Python-MIP, CGC are implemented extending the ConstrsGenerator class. The following example implements the previous cut separation algorithm as a ConstrsGenerator class and includes it as a cut generator for the branch-and-cut solver engine. The method that needs to be implemented in this class is the generate_constrs() procedure. This method receives as parameter the object model of type Model. This object must be used to query the fractional values of the model vars, using the x property. Other model properties can be queried, such as the problem constraints (constrs). Please note that, depending on which solver engine you use, some variables/constraints from the original model may have been removed in the pre-processing phase. Thus, direct references to the original problem variables may be invalid. The method translate() (line 15) translates references of variables from the original model to references of variables in the model received in the callback procedure. Whenever a violated inequality is discovered, it can be added to the model using the += operator (line 31). In our example, we temporarily store the generated cuts in a CutPool object (line 25) to discard repeated cuts that eventually are found.



In [1]:
from typing import List, Tuple
from random import seed, randint
from itertools import product
from math import sqrt
import networkx as nx
from mip import Model, xsum, BINARY, minimize, ConstrsGenerator, CutPool


class SubTourCutGenerator(ConstrsGenerator):
    """Class to generate cutting planes for the TSP"""
    def __init__(self, Fl: List[Tuple[int, int]], x_, V_):
        self.F, self.x, self.V = Fl, x_, V_

    def generate_constrs(self, model: Model, depth: int = 0, npass: int = 0):
        xf, V_, cp, G = model.translate(self.x), self.V, CutPool(), nx.DiGraph()
        for (u, v) in [(k, l) for (k, l) in product(V_, V_) if k != l and xf[k][l]]:
            G.add_edge(u, v, capacity=xf[u][v].x)
        for (u, v) in F:
            val, (S, NS) = nx.minimum_cut(G, u, v)
            if val <= 0.99:
                aInS = [(xf[i][j], xf[i][j].x)
                        for (i, j) in product(V_, V_) if i != j and xf[i][j] and i in S and j in S]
                if sum(f for v, f in aInS) >= (len(S)-1)+1e-4:
                    cut = xsum(1.0*v for v, fm in aInS) <= len(S)-1
                    cp.add(cut)
                    if len(cp.cuts) > 256:
                        for cut in cp.cuts:
                            model += cut
                        return
        for cut in cp.cuts:
            model += cut


n = 30  # number of points
V = set(range(n))
seed(0)
p = [(randint(1, 100), randint(1, 100)) for i in V]  # coordinates
Arcs = [(i, j) for (i, j) in product(V, V) if i != j]

# distance matrix
c = [[round(sqrt((p[i][0]-p[j][0])**2 + (p[i][1]-p[j][1])**2)) for j in V] for i in V]

model = Model()

# binary variables indicating if arc (i,j) is used on the route or not
x = [[model.add_var(var_type=BINARY) for j in V] for i in V]

# continuous variable to prevent subtours: each city will have a
# different sequential id in the planned route except the first one
y = [model.add_var() for i in V]

# objective function: minimize the distance
model.objective = minimize(xsum(c[i][j]*x[i][j] for (i, j) in Arcs))

# constraint : leave each city only once
for i in V:
    model += xsum(x[i][j] for j in V - {i}) == 1

# constraint : enter each city only once
for i in V:
    model += xsum(x[j][i] for j in V - {i}) == 1

# (weak) subtour elimination
# subtour elimination
for (i, j) in product(V - {0}, V - {0}):
    if i != j:
        model += y[i] - (n+1)*x[i][j] >= y[j]-n

# no subtours of size 2
for (i, j) in Arcs:
    model += x[i][j] + x[j][i] <= 1

# computing farthest point for each point, these will be checked first for
# isolated subtours
F, G = [], nx.DiGraph()
for (i, j) in Arcs:
    G.add_edge(i, j, weight=c[i][j])
for i in V:
    P, D = nx.dijkstra_predecessor_and_distance(G, source=i)
    DS = list(D.items())
    DS.sort(key=lambda x: x[1])
    F.append((i, DS[-1][0]))

model.cuts_generator = SubTourCutGenerator(F, x, V)
model.optimize()

print('status: %s route length %g' % (model.status, model.objective_value))

Welcome to the CBC MILP Solver 
Version: devel 
Build Date: Nov 15 2020 

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 1307 (-435) rows, 899 (-31) columns and 5046 (-870) elements
Clp1000I sum of infeasibilities 4.34343e-05 - average 3.32321e-08, 703 fixed columns
Coin0506I Presolve 942 (-365) rows, 194 (-705) columns and 2265 (-2781) elements
Clp0029I End of values pass after 194 iterations
Clp0014I Perturbing problem by 0.001% of 9.7457489 - largest nonzero change 5.2150316e-05 ( 0.0010336379%) - largest zero change 2.7270429e-05
Clp0000I Optimal - objective value 384
Clp0000I Optimal - objective value 384
Coin0511I After Postsolve, objective 384, infeasibilities - dual 0 (0), primal 0 (0)
Clp0000I Optimal - objective value 384
Clp0000I Optimal - objective value 384
Clp0000I Optimal - objective value 384
Coin0511I After Postsolve, objective 384, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 384 - 0 ite